In [ ]:
! pip install beautifulsoup4

In [ ]:
! pip install selenium

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

import time

In [ ]:
# We open Chrome browser in incognito mode
service = ChromeService(executable_path=ChromeDriverManager().install())

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

driver = webdriver.Chrome(service=service, chrome_options=chrome_options)

In [ ]:
# Link to the desired page to be scraped
driver.get('https://foxhugh.com/interview-questions/list-of-interview-questions/');

# Save all the content in the "soup" variable
content = driver.page_source
soup = BeautifulSoup(content)

#List to store de questions
allQuestions = [] 

#The content we want is in a "ol" type layer 
new = soup.ol

#Get the content
parent=new.findAll('li')

#Iterate the content 1 by 1
for child in parent:
    print(child.string)
    allQuestions.append(child.string)

In [ ]:
# Loop to delete duplicated and None questions (in case that there are)
questions = []
 
for element in allQuestions:
    if element != None :
        if element not in questions:
            questions.append(element)

print(questions)

In [ ]:
#We will iterate over all the saved questions and translate them all, we have 1003
print(len(questions))
print(len(allQuestions))

In [ ]:
print(questions)

In [ ]:
#Save all the questions in a CSV
df = pd.DataFrame({'Interviewer Questions':questions}) 
df.to_csv('Interviewer_Questions.csv', index=False, encoding='utf-8')

In [ ]:
from selenium.webdriver.common.by import By
import pyperclip

#Now lets go to the google traductor page
driver.get('https://translate.google.es/?hl=es&sl=en&tl=es&op=translate');
time.sleep(5) # Let the user actually see something!

In [ ]:
#Accept google cookies
driver.find_element(By.XPATH, "//span[normalize-space()='Acepto']").click()

In [ ]:
# Global list to save the translated questions
questions_translated = []

In [ ]:
len(questions)

In [ ]:
i=0
stop=0

# Get the search box and clean it 
search_box = driver.find_element(By.CLASS_NAME, "er8xn")
search_box.clear()

def questions_traductor(index):
    i=index
    last_answer=[]
    translation=[]
    
    while i < len(questions):
        #if the program fails, stop will save the index and it will restart at the point where failed
        stop=i
        
        # Print the position and question to see if everything goes well
        print(i)
        print(questions[i])        
        
        # If the question actually exists (errors or blank lines could be imported in the scrapping)
        if questions[i] is not None:

            # Get the search_box to send the question
            try:
                search_box = driver.find_element(By.CLASS_NAME, "er8xn")
            except:
                questions_traductor(stop)

            # Questions sended to the translator
            search_box.send_keys(questions[i])
            
            # Counter
            iteration=0
            
            # Sleep needed to let the traductor respond, if the actual question and the last one are the same
            # we must keep waiting, this solution is better than waiting a constant amount of time 
            while (last_answer==translation):
                
                #if the program is stuck in here (>5 iterations) its probably because the 2 original sentences are similar
                #and only change a comma or any other punctuaction mark, but that is ignored by the translator because
                #the destiny language doesnt need it so the program detects that 2 very similar sentences are the same
                #and keeps trying to make a better traduction
                iteration=iteration+1
                if iteration>5:
                    break
                
                print("Equals, more time needed for the translator to respond")
                time.sleep(1)
                try:
                    # find the copy translation button and click on it
                    driver.find_element(By.XPATH, "//button[@aria-label='Copiar traducción']//div[@class='VfPpkd-Bz112c-RLmnJb']").click()
                    translation = []
                    translation=pyperclip.paste()
                except:
                    questions_traductor(stop)
            
            print("Last question", last_answer)
            print("Current question", translation)

            # Append the translation
            questions_translated.append(translation)

            # Clear the search box to keep translating
            search_box.clear()
            
            # The actual answer will be the last in the next iteration so we can check and wait till traductor responds
            last_answer=translation
            last=questions[i]
            
            
        
        # Next question, doesnt matter if the question before existed or not
        i=i+1

In [ ]:
questions_traductor(stop)

In [ ]:
#Save all the questions in a CSV
df = pd.DataFrame({'Interviewer Questions Translated':questions_translated}) 
df.to_csv('Interviewer_Questions_Translated_v2.csv', index=False, encoding='utf-8')

In [ ]:
# Check that the lenght is the same as the original questions list
len(questions_translated)

In [ ]:
print(questions_translated)

In [ ]:
#Loop to check if there is any repeated question or none

last=[]

for element in questions_translated:
    if element != None :
        if element not in last:
            last.append(element)
        else:
            print("REPEATED")
    else:
        print("NONE")

In [ ]:
#Close the driver
driver.quit()